# Importar Access .mdb

## Modulos requeridos

In [ ]:
import os
import pyodbc
import sqlite3

### Access Drivers

In [ ]:
[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

### Working Directory

In [ ]:
WD = os.getcwd()
WD

## Datos iniciales

In [ ]:
# set up some constants
MDB = WD + '\\Slave.mdb'
DRV = '{Microsoft Access Driver (*.mdb, *.accdb)}'
PWD = ''
print(f'Dirección: {MDB} - Driver: {DRV}')

## Conectar a la DB

In [ ]:
#con = pyodbc.connect('DRIVER={};DBQ={}'.format(DRV,MDB))
#cur = con.cursor()
""" 
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=D:\Datos INVICO\Python INVICO\Lectura archivos\read_slave\Slave.mdb;'
    ) 

 """
conn_str = (
    f'DRIVER={DRV};DBQ={MDB};'
    )  

con_mdb = pyodbc.connect(conn_str)
cur_mdb = con_mdb.cursor()

## Query

### Tablas

In [ ]:
for table_info in cur_mdb.tables(tableType='TABLE'):
    print(table_info.table_name)

### Registros de una tabla específica

#### a) Tabla Precarizados

In [ ]:
SQL = 'SELECT * FROM PRECARIZADOS;' # your query goes here
rows_precarizados_mdb = cur_mdb.execute(SQL).fetchall()
list(rows_precarizados_mdb[1])

#### b) Tabla Liquidación Honorarios

In [ ]:
SQL = 'SELECT * FROM LIQUIDACIONHONORARIOS;' # your query goes here
rows_honfact_mdb = cur_mdb.execute(SQL).fetchall()
list(rows_honfact_mdb[1])


## Creamos / actualizamos slave.sqlite

### Datos iniciales de la conexión

In [ ]:
SQLITE_DB = 'slave_test.sqlite'

con_sqlite = sqlite3.connect(SQLITE_DB)
cur_sqlite = con_sqlite.cursor()

### Creación de tablas

#### a) Tabla Precarizados

In [ ]:
con_sqlite.execute('''
DROP TABLE IF EXISTS factureros
''')
con_sqlite.commit()

con_sqlite.execute('''
CREATE TABLE if not exists factureros (
    id_factureros INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL, 
    razon_social TEXT NOT NULL, 
    estructura TEXT NOT NULL, 
    partida TEXT NOT NULL)
''')
con_sqlite.commit()

for item in rows_precarizados_mdb:
    item = list(item)
    item[1] = (item[1][0:2] + '-00' + item[1][2:])
    cur_sqlite.execute("INSERT INTO factureros (razon_social, estructura, partida) VALUES(?,?,?)", item)
con_sqlite.commit()

# i = 0
# for item in rows_precarizados_mdb:
#     i += 1
#     item = list(item)
#     item.insert(0, i)
#     cur_sqlite.execute("INSERT INTO factureros VALUES(?,?,?,?)", item)
# con_sqlite.commit()

#### b) Tabla Honorarios Factureros

In [ ]:
con_sqlite.execute('''
DROP TABLE IF EXISTS honorarios
''')
con_sqlite.commit()

con_sqlite.execute('''
CREATE TABLE if not exists honorarios (
    id_honorarios INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL, 
    fecha INTEGER, 
    razon_social TEXT, 
    sellos REAL, 
    seguro REAL, 
    nro_entrada TEXT, 
    tipo TEXT, 
    importe_bruto REAL, 
    iibb REAL, 
    libramiento_pago REAL, 
    otras_retenciones REAL, 
    anticipo REAL, 
    mutual REAL,
    estructura TEXT, 
    partida TEXT,
    embargo REAL)
''')
con_sqlite.commit()

# i = 0
for item in rows_honfact_mdb:
    # i += 1
    item = list(item)
    # item.insert(0, i)
    #item[0] = datetime.timestamp(item[0])
    item[2] = float(item[2])
    item[3] = float(item[3])
    item[6] = float(item[6])
    item[7] = float(item[7])
    item[8] = float(item[8])
    item[9] = float(item[9])
    item[10] = float(item[10])
    item[11] = float(item[11])
    item[12] = (item[12][0:2] + '-00' + item[12][2:])
    # cur_sqlite.execute("INSERT INTO factureros (nombre_completo, actividad, partida) VALUES(?,?,?)", item)
    cur_sqlite.execute('''
    INSERT INTO honorarios (
        fecha, 
        razon_social, 
        sellos,    
        seguro,
        nro_entrada,    
        tipo,    
        importe_bruto,
        iibb, 
        libramiento_pago,    
        otras_retenciones,
        anticipo, 
        mutual,
        estructura, 
        partida, 
        embargo
        ) 
    VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,0)
    ''', item)
con_sqlite.commit()

#### c) Desagregamos tabla honorarios

In [ ]:
#Creamos tabla comprobantes_siif
con_sqlite.execute('''
DROP TABLE IF EXISTS comprobantes_siif
''')
con_sqlite.commit()

con_sqlite.execute('''
CREATE TABLE if not exists comprobantes_siif (
    nro_entrada TEXT PRIMARY KEY UNIQUE NOT NULL, 
    fecha INTEGER, 
    tipo TEXT)
''')
con_sqlite.commit()

#Insertamos registros en la tabla comprobantes_siif
con_sqlite.execute('''
INSERT INTO comprobantes_siif
    SELECT DISTINCT nro_entrada, fecha, tipo
    FROM honorarios
''')
con_sqlite.commit()

#Eliminamos columnas en honorarios
con_sqlite.execute('''
DROP TABLE IF EXISTS temporal
''')
con_sqlite.commit()

con_sqlite.execute('''
CREATE TABLE if not exists temporal (
    id_honorarios INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL, 
    nro_entrada TEXT,
    razon_social TEXT, 
    importe_bruto REAL, 
    iibb REAL,
    libramiento_pago REAL, 
    sellos REAL, 
    seguro REAL,  
    otras_retenciones REAL, 
    anticipo REAL, 
    mutual REAL,
    embargo REAL, 
    estructura TEXT, 
    partida TEXT,
    FOREIGN KEY(nro_entrada) REFERENCES comprobantes_siif(nro_entrada) ON DELETE CASCADE)
''')
con_sqlite.commit()

con_sqlite.execute('''
INSERT INTO temporal
    SELECT id_honorarios, nro_entrada, razon_social, importe_bruto, 
    iibb, libramiento_pago, sellos, seguro, otras_retenciones, 
    anticipo, mutual, embargo,
    estructura, partida
    FROM honorarios
''')
con_sqlite.commit()

con_sqlite.execute('''
DROP TABLE IF EXISTS honorarios
''')
con_sqlite.commit()

con_sqlite.execute('''
ALTER TABLE temporal RENAME TO honorarios
''')
con_sqlite.commit()

## Cerrar conexión

In [ ]:
cur_mdb.close()
con_mdb.close()

cur_sqlite.close()
con_sqlite.close()